# 5. Composição de Múltiplas Visualizações
---

Ao visualizar vários campos de dados diferentes, podemos ficar tentados a usar tantos canais de codificação visual quanto pudermos: `x`, `y`, `color`, `size`, `shape` e assim por diante. No entanto, à medida que o número de canais de codificação aumenta, um gráfico pode rapidamente tornar-se desordenado e difícil de ler. Uma alternativa para “sobrecarregar” um único gráfico é, em vez disso, compor vários gráficos de uma forma que facilite comparações rápidas.

Neste capítulo, examinaremos uma variedade de operações para composição de múltiplas visualizações:



*   camada: colocar gráficos compatíveis diretamente uns sobre os outros,
*   faceta: particionar dados em múltiplos gráficos, organizados em linhas ou colunas,
*   concatenação: posicionar gráficos arbitrários em um layout compartilhado, e
*   repetição: pegar uma especificação de gráfico base e aplica-lá a vários campos de dados.




Veremos então como essas operações formam uma álgebra de composição de visualizações, em que as operações podem ser combinadas para construir uma variedade de exibições complexas de múltiplas visualizações.

Este capítulo faz parte do [data visualization curriculum ](https://github.com/uwdata/visualization-curriculum).

In [1]:
import pandas as pd
import altair as alt

## 5.1 Dados Meteorológicos
---

Estaremos visualizando estatísticas meteorológicas para as cidades americanas de Seattle e Nova York. Vamos carregar o conjunto de dados e dar uma olhada nas primeiras e últimas 10 linhas:

In [2]:
weather = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/weather.csv'

In [3]:
df = pd.read_csv(weather)
df.head(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
0,Seattle,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,Seattle,2012-01-02,10.9,10.6,2.8,4.5,rain
2,Seattle,2012-01-03,0.8,11.7,7.2,2.3,rain
3,Seattle,2012-01-04,20.3,12.2,5.6,4.7,rain
4,Seattle,2012-01-05,1.3,8.9,2.8,6.1,rain
5,Seattle,2012-01-06,2.5,4.4,2.2,2.2,rain
6,Seattle,2012-01-07,0.0,7.2,2.8,2.3,rain
7,Seattle,2012-01-08,0.0,10.0,2.8,2.0,sun
8,Seattle,2012-01-09,4.3,9.4,5.0,3.4,rain
9,Seattle,2012-01-10,1.0,6.1,0.6,3.4,rain


In [4]:
df.tail(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
2912,New York,2015-12-22,4.8,15.6,11.1,3.8,fog
2913,New York,2015-12-23,29.5,17.2,8.9,4.5,fog
2914,New York,2015-12-24,0.5,20.6,13.9,4.9,fog
2915,New York,2015-12-25,2.5,17.8,11.1,0.9,fog
2916,New York,2015-12-26,0.3,15.6,9.4,4.8,drizzle
2917,New York,2015-12-27,2.0,17.2,8.9,5.5,fog
2918,New York,2015-12-28,1.3,8.9,1.7,6.3,snow
2919,New York,2015-12-29,16.8,9.4,1.1,5.3,fog
2920,New York,2015-12-30,9.4,10.6,5.0,3.0,fog
2921,New York,2015-12-31,1.5,11.1,6.1,5.5,fog


Criaremos exibições de multivisualizações para examinar o clima dentro e fora das cidades.

## 5.2 Camada
---

Uma das formas mais comuns de combinar vários gráficos é *sobrepor* marcas umas sobre as outras. Se os domínios das escalas subjacentes forem compatíveis, podemos mesclá-los para formar *eixos compartilhados*. Se uma das codificações em `x` ou `y` não for compatível, podemos, em vez disso, criar um gráfico de *eixos duplos*, que sobrepõe marcas usando escalas e eixos separados.

### 5.2.1 Eixos Compartilhados

Vamos começar plotando as temperaturas médias mínima e máxima por mês:

In [5]:
alt.Chart(weather).mark_area().encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q')
)

alt.Chart(...)

*O gráfico nos mostra os intervalos de temperatura para cada mês ao longo de todo o nosso conjunto de dados. No entanto, isso pode ser bastante enganoso, pois agrega as medições de Seattle e Nova York!*

Vamos subdividir os dados por local usando uma codificação de cor e também ajustar a opacidade das marcas para acomodar as áreas sobrepostas:

In [6]:
alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

alt.Chart(...)

*Podemos ver que Seattle tem um clima mais ameno: é mais quente no inverno e mais fresco no verão.*

Neste caso, criamos um gráfico em camadas sem recursos especiais, apenas subdividindo as marcas de área por cor. Embora o gráfico acima nos mostre os intervalos de temperatura, também podemos querer enfatizar o ponto médio desse intervalo.

Vamos criar um gráfico de linhas mostrando a temperatura média intermediária. Para isso, usaremos uma transformação de cálculo (`calculate`) para determinar os pontos médios entre as temperaturas mínimas e máximas diárias:

In [7]:
alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.Chart(...)

*Observação*: note o uso de `+datum.temp_min` dentro da transformação de cálculo. Como estamos carregando os dados diretamente de um arquivo CSV sem nenhuma instrução especial de análise, os valores de temperatura podem estar internamente representados como strings. Adicionar o `+` antes do valor força sua interpretação como um número.

Agora, queremos combinar esses gráficos sobrepondo as linhas dos pontos médios às áreas de intervalo. Usando a sintaxe `chart1 + chart2`, podemos especificar que queremos um novo gráfico em camadas em que `chart1` é a primeira camada e `chart2` é uma segunda camada desenhada por cima:

In [8]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid

alt.LayerChart(...)

*Agora temos um gráfico em várias camadas! No entanto, o título do eixo y (embora informativo) ficou um pouco longo e desorganizado...*

Vamos personalizar nossos eixos para melhorar a aparência do gráfico. Se definirmos um título personalizado para o eixo em uma das camadas, ele será automaticamente usado como título compartilhado para todas as camadas:

In [9]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid

alt.LayerChart(...)

*O que acontece se ambas as camadas tiverem títulos personalizados para o eixo? Modifique o código acima para descobrir...*

Acima usamos o operador `+`, uma forma conveniente de escrever o método de camadas (`layer`) do Altair. Podemos gerar um gráfico em camadas idêntico usando diretamente o método `layer`:

In [10]:
alt.layer(tempMinMax, tempMid)

alt.LayerChart(...)

Observe que a ordem dos elementos em uma camada é importante, pois as camadas subsequentes serão desenhadas sobre as anteriores. *Tente trocar a ordem dos gráficos nas células acima. O que acontece? (Dica: preste atenção na cor das marcas de linha (`line`).)*

### 5.2.2 Gráficos de Eixo Duplo

*Seattle tem a reputação de ser uma cidade chuvosa. Será que isso é merecido?*

Vamos analisar a precipitação junto com a temperatura para entender melhor. Primeiro, vamos criar um gráfico básico que mostra a precipitação média mensal em Seattle:

In [11]:
alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.Chart(...)

Para facilitar a comparação com os dados de temperatura, vamos criar um novo gráfico em camadas (_layered chart_). Aqui está o que acontece se tentarmos sobrepor os gráficos da mesma forma que fizemos anteriormente:

In [12]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip)

alt.LayerChart(...)

*Os valores de precipitação utilizam um intervalo muito menor no eixo y do que as temperaturas!*

Por padrão, gráficos em camadas (_layered charts_) utilizam um `domínio compartilhado`: os valores para o eixo x ou y são combinados entre todas as camadas para determinar uma extensão compartilhada. Esse comportamento padrão assume que os valores das camadas têm as mesmas unidades. No entanto, isso não se aplica a este exemplo, pois estamos combinando valores de temperatura (graus Celsius) com valores de precipitação (polegadas)!

Se quisermos usar escalas diferentes para o eixo y, precisamos especificar como desejamos que o Altair *resolva* os dados entre as camadas. Neste caso, queremos que os domínios das escalas (`scale`) do eixo y sejam independentes (`independent`), em vez de usar um domínio compartilhado (`shared`). O objeto `Chart` gerado por um operador de camada inclui um método `resolve_scale` (resolve escala) com o qual podemos especificar a resolução desejada.

In [13]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip).resolve_scale(y='independent')

alt.LayerChart(...)

*Agora podemos ver que o outono é a estação mais chuvosa em Seattle (com pico em novembro), complementado pelos verões secos.*

Você pode ter notado certa redundância nas especificações do nosso gráfico acima: ambos usam o mesmo conjunto de dados e o mesmo filtro para exibir apenas Seattle. Se vocês quiser, você pode simplificar um pouco o código fornecendo os dados e a transformação de filtro para o gráfico em camadas (_layered chart_) de nível superior. As camadas individuais irão herdar os dados se não tiverem suas próprias definições de dados.

In [14]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart().mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip, data=weather).transform_filter(
  'datum.location == "Seattle"'
).resolve_scale(y='independent')

alt.LayerChart(...)

Embora gráficos de eixos duplos (_dual-axis charts_) possam ser úteis, *eles são frequentemente propensos a interpretações equivocadas*, pois as diferentes unidades e escalas dos eixos podem ser desproporcionais. Sempre que possível, você pode considerar transformações que mapeiem diferentes campos de dados para unidades compartilhadas, como, por exemplo, exibir [quantis](https://en.wikipedia.org/wiki/Quantile) ou mudanças percentuais relativas.

## 5.3 Faceta
---

*Facetar* envolve subdividir um dataset em grupos e criar um gráfico separado para cada grupo. Nos últimos capítulos, aprendemos como criar tabelas facetadas usando os canais de codificação `linha` e `coluna`. Vamos primeiro revisar esses canais e então mostrar como são instâncias de um operador mais geral: a `faceta`.

Comecemos com um histograma básico acerca dos níveis máximos da temperatura de Seattle:

In [15]:
alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q')
)

alt.Chart(...)

_Como o perfil de temperaturas buda com base no tempo de um dado dia - isto é, se chovia, garoava, nevava, fazia sol ou tempo nublado?_

Vamos usar o canal de codificação `coluna` para facetar os dados por tipo de tempo atmosférico. Podemos também usar a `cor` como codificação redundante, usando um intervalo customizado:

In [16]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors),
  alt.Column('weather:N')
).properties(
  width=150,
  height=150
)

alt.Chart(...)

_Para a surpresa de ninguém, esses raros dias de neve centralizam as temperaturas mais baixas, seguidos por dias chuvosos e nublados. Dias de sol são mais quentes e, apesar dos estereótipos de Seattle, são os mais frequentes. Apesar de que, como qualquer morador de Seattle pode te informar, a garoa vem e volta a qualquer momento e a despeito da temperatura!_

Adicionalmente aos canais de codificação `linha` e `coluna`, uma definição de tabelas, podemos usar definições básicas de tabela para aplicar facetamento através de um operador explícito de `faceta`.

Vamos recriar a tabela acima, mas dessa vez usando `faceta`. Começamos com algumas definições básicas de histograma, mas removemos a fonte das informações, filtro de transformação e canal coluna. Podemos então chamar o método `faceta` passando os dados e especificando que se deve facetar colunas de acordo com o campo `clima`. O método `faceta` aceita tanto `linha` quanto `coluna` como  argumentos. Ambos podem ser usados para criar uma grade 2d de gráficos facetados.

Por fim, incluímos nossos filtros de transformação, os apliccando ao gráfico da faceta superior (_faceted plots_). Poderíamos, é claro, aplicar o filtro de transformação à definição de histograma apresentada no início, mas isso seria sensivelmente menos eficiente. Melhor que filtrar valores de "Nova York" dentro de cada célula de faceta, aplicar o filtro ao gráfico facetado (_faceted plot_) permite que o Vega-Lite saiba que podemos filtrar esses valores antecipadamente, antes da divisão das facetas.

In [17]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart().mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors)
).properties(
  width=150,
  height=150
).facet(
  data=weather,
  column='weather:N'
).transform_filter(
  'datum.location == "Seattle"'
)

alt.FacetChart(...)


Dado todo o código extra acima, por quereríamos usar um operador explícito para `faceta`? Para gráficos básicos, deveríamos certamente usar os canais de codificação `coluna` ou `linha` sempre que possível. Usar o operador explícito `faceta`, entretanto, é útil se quisermos facetar visualizações compostas como gráficos em camadass.

Vamos revisitar nossas plotagens de temperatura em camadas usadas mais cedo. Ao invés de plotar dados de Nova York e Seattle no mesmo gráfico, os quebraremos em diferentes facetas. As definições de gráfico individual são basicamente as mesmas de antes: um gráfico de área e um gráfico de linha. A única diferença é que dessa vez não passaremos o dados diretamente para os construtores do gráfico; vamos esperar e passá-los para o operador faceta posteriormente. Podemos construir camadas de gráficos como antes e só então chamar o `faceta` para o objeto gráfico em camada, passando o dado e especificando facetas `coluna` com base no campo `localização`:

In [18]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
)

alt.FacetChart(...)

Os gráficos facetados que vimos até agora usam os mesmos domínios de escala e eixos através nas células facetadas. Esse padrão de usar escalas e eixos *compartilhados* ajuda a auxiliar na comparação precisa de valores. No entanto, em alguns casos, você pode querer dimensionar cada gráfico independentemente, por exemplo, se o intervalo de valores nas células diferir significativamente.

Semelhante aos gráficos em camadas, os gráficos facetados também oferecem suporte à _resolução_ para escalas ou eixos independentes nos gráficos. Vamos ver o que acontece se chamarmos o método `resolve_axis` para solicitar eixos y `independentes`:

In [19]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_axis(y='independent')

alt.FacetChart(...)

_O gráfico acima não parece ter sofrido grandes mudanças, mas o plot de Seattle agora inclui seu próprio eixo._

E se ao invés disso chamássemos `resolve_scale` para solucionar os domínios de escala subjacentes?

In [20]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_scale(y='independent')


alt.FacetChart(...)

_Vemos agora células facetadas com diferentes eixos e domínios de escala. Nesse caso, usar escalas independentes parece uma ideia ruim! Os domínios não tem diferença que o justifica, e o observador pode ser induzido ao erro de achar que Nova York e Seattle tem temperaturas máximas semelhantes no verão._

Para tomar um clicê emprestado: só poruque você *pode* fazer algo, não significa que *deveria*...

## 5.4 Concatenação
---

A técnica de faceting cria [múltiplos gráficos pequenos](https://en.wikipedia.org/wiki/Small_multiple) que mostram subdivisões separadas do dado. No entanto, podemos desejar criar uma exibição de multivisualização com diferentes perspectivas do *mesmo* conjunto de dados (não mais subconjuntos) ou visualizações que envolvem conjuntos de dados *distintos*.

O Altair oferece operadores de *concatenação* para combinar gráficos arbitrários em um único gráfico concatenado. O operador `hconcat` (abreviando, `|` ) realiza uma concatenação horizontal, enquanto o operador `vconcat` (abreviando, `&`) realiza uma concatenação vertical.

Vamos começar com um gráfico de linha simples, mostrando a média da temperatura máxima por mês em Nova York e Seattle, conforme vimos anteriormente:


In [21]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(temp_max):Q'),
  color='location:N'
)

alt.Chart(...)

*E se quisermos comparar não apenas a temperatura ao longo do tempo, mas também os níveis de precipitação e ventos?*


Vamos criar um gráfico concatenado composto por três visualizações. Vamos começar definindo um gráfico “base”  que contém todos os aspectos que devem ser compartilhados pelas três visualizações. Podemos, então, modificar esta base para criar variantes customizadas, com diferentes codificações do eixo y para as variáveis de temperatura máxima (`temp_max`), precipitação (`precipitation`) e ventos (`wind`). A seguir, concatenamos as visualizações usando o operador abreviado da barra vertical ( `|` ):


In [22]:
base = alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  color='location:N'
).properties(
  width=240,
  height=180
)

temp = base.encode(alt.Y('average(temp_max):Q'))
precip = base.encode(alt.Y('average(precipitation):Q'))
wind = base.encode(alt.Y('average(wind):Q'))

temp | precip | wind

alt.HConcatChart(...)

Poderíamos também utilizar o método mais explícito `alt.hconcat()` no lugar da barra vertical `|` . *Tente reescrever o código acima usando* `hconcat`.

A concatenação vertical funciona de forma similar à concatenação horizontal. *Usando o operador `&` (ou o método* `alt.vconcat`)*, modifique o código para ordenar as visualizações verticalmente ao invés de horizontalmente.*

Finalmente, note que as concatenações horizontais e verticais podem ser combinadas. *O que acontece se você escrever algo como* `(temp | precip) & wind` *?*

*Observação:* Note a importância desses parênteses… O que aconteceria se você os removesse? Mantenha em mente que esses operadores sobrecarregados ainda estão sujeitos às [regras de precedência de operadores do Python](https://docs.python.org/3/reference/expressions.html#operator-precedence), e assim, a concatenação vertical com `&` vai ter preferência sobre a concatenação horizontal com `|` !

Como veremos mais tarde, os operadores de concatenação permitem combinar quaisquer gráficos em um dashboard de multivisualização!



## 5.5 Repetição
---


Os operadores de concatenação são genéricos, permitindo a composição de gráficos arbitrários. No entanto, o exemplo acima ainda possui palavras demais: temos três gráficos muito semelhantes, mas precisamos defini-los separadamente e depois concatená-los.

Para casos em que apenas uma ou duas variáveis estão mudando, o operador `repeat` proporciona um atalho útil para criar múltiplos gráficos. Dada uma especificação de *modelo* com algumas variáveis livres, o operador `repeat` criará um gráfico para cada atribuição especificada dessas variáveis.

Vamos recriar o exemplo de concatenação acima usando o operador `repeat`. O único aspecto que difere entre as visualizações é a escolha da variável para a codificação do eixo `y`. Para criar uma especificação de modelo, podemos usar a *variável repetitiva* `alt.repeat('column')` como o nosso eixo y. Esse código simplesmente indica que queremos usar a variável atribuída à coluna repetida, a qual organiza os gráficos repetidos na direção horizontal. (Como o método fornece apenas o nome do campo, temos que especificar o tipo dos dados separadamente com `type='quantitative'` .)

Em seguida, invocamos o método `repeat`, passando os nomes dos campos de dados para cada coluna:


In [23]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T',title=None),
  alt.Y(alt.repeat('column'), aggregate='average', type='quantitative'),
  color='location:N'
).properties(
  width=240,
  height=180
).repeat(
  column=['temp_max', 'precipitation', 'wind']
)

alt.RepeatChart(...)

*A repetição pode ser feita tanto nas colunas quanto nas linhas. O que acontece se você modificar o código acima para usar linhas (*`row`*) ao invés de colunas (*`column`*)?*


Podemos também usar a repetição de linha (`row`) e de coluna (`column`) juntas! Uma visualização comum para exploração de análise de dados é a [matriz de gráfico de dispersão](https://pt.wikipedia.org/wiki/Gr%C3%A1fico_de_dispers%C3%A3o) (ou SPLOM, abreviação do termo em inglês 'scatter plot matrix'). Dada uma coleção de variáveis para inspecionar, a SPLOM proporciona uma grade com todos os gráficos pareados dessas variáveis, nos permitindo enxergar possíveis correlações.

Vamos usar o operador `repeat` para criar uma SPLOM para os campos `temp_max`, `precipitation` e `wind`. Primeiramente, vamos criar a nossa especificação de modelo, com variáveis repetidas para os campos do eixo x e do eixo y.  A seguir, invocamos `repeat`, passando vetores de nomes de campos para usar tanto na linha (`row`) quanto na coluna (`column`). O Altair vai, então, gerar o [ produto cruzado (ou o produto Cartesiano)](https://pt.wikipedia.org/wiki/Produto_cartesiano) para criar o espaço completo de gráficos repetidos:


In [24]:
alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=150,
  height=150
).repeat(
  data=weather,
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
).transform_filter(
  'datum.location == "Seattle"'
)

alt.RepeatChart(...)

*Analisando essas visualizações, parece que não há uma correlação forte entre precipitação e ventos, apesar de notarmos que eventos com fortes ventos e precipitação ocorrem em intervalos similares de máxima temperatura (~5-15° C). Contudo, essa observação não é muito surpreendente: relembrando o histograma feito no começo da seção de facet, podemos ver claramente que os dias com máximas temperaturas no intervalo de 5-15°C são os mais frequentes.*

*Modifique o código acima para conseguir uma melhor compreensão da repetição de gráficos. Tente adicionar outra variável (*`temp_min`*) para a SPLOM. O que acontece se você rearranjar a ordem dos campos em algum dos parâmetros de linha (*`row`*) ou coluna (*`column`*) para o operador* `repeat`*?*

*Finalmente, para realmente apreciar tudo que o operador*`repeat` *pode oferecer, reserve um momento para imaginar como você pode recriar a SPLOM acima usando apenas* `hconcat` *e* `vconcat`*!*


## 5.6 Álgebra de Composição de Visualizações
---

Juntos, os operados `layer`, `facet`, `concat` e `repeat` formam uma _álgebra de composição de visualizações_: diversos operadores podem ser combinados para construir uma variedade de visualizações múltiplas.

Como exemplo, vamos começar com dois gráficos básicos: um histograma e uma linha simples (uma única marca `rule`) mostrando uma média global.

In [25]:
basic1 = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_bar().encode(
  alt.X('month(date):O'),
  alt.Y('average(temp_max):Q')
)

basic2 = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_rule(stroke='firebrick').encode(
  alt.Y('average(temp_max):Q')
)

basic1 | basic2

alt.HConcatChart(...)

Podemos então combinar os dois gráficos usando um operador `layer` e, em seguida, ` repeat`, que repetiria esse gráfico em camadas para mostrar histogramas com médias sobrepostas para vários campos:

In [26]:
alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Month'),
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  )
).properties(
  width=200,
  height=150
).repeat(
  data=weather,
  column=['temp_max', 'precipitation', 'wind']
).transform_filter(
  'datum.location == "Seattle"'
)

alt.RepeatChart(...)

Se focarmos apenas nos operadores de composição de múltiplas visualizações, o modelo para a visualização acima é:

```
repeat(column=[...])
|- layer
   |- basic1
   |- basic2
```

Agora, vamos explorar como podemos aplicar todos os operadores em um [painel](https://en.wikipedia.org/wiki/Dashboard_(computing)) final que fornece uma visão geral do clima de Seattle. Combinaremos os displays SPLOM e histograma facetado das seções anteriores com os histogramas repetidos acima:

In [27]:
splom = alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=125,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
)

dateHist = alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Month'),
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  )
).properties(
  width=175,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind']
)

tempHist = alt.Chart(weather).mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=alt.Scale(
    domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
    range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
  ))
).properties(
  width=115,
  height=100
).facet(
  column='weather:N'
)

alt.vconcat(
  alt.hconcat(splom, dateHist),
  tempHist,
  data=weather,
  title='Seattle Weather Dashboard'
).transform_filter(
  'datum.location == "Seattle"'
).resolve_legend(
  color='independent'
).configure_axis(
  labelAngle=0
)

alt.VConcatChart(...)

O modelo de composição completo para este painel é:
```
vconcat
|- hconcat
|  |- repeat(row=[...], column=[...])
|  |  |- splom base chart
|  |- repeat(row=[...])
|     |- layer
|        |- dateHist base chart 1
|        |- dateHist base chart 2
|- facet(column='weather')
   |- tempHist base chart
```
Ufa! O painel também inclui algumas personalizações para melhorar o layout:

* Ajustamos o gráfico com as propriedades `width` e `height` para auxiliar o alinhamento e garantir que a visualização completa caiba na tela.

* Adicionamos `resolve_legend(color='independent')` para garantir que a legenda de cores seja associada diretamente aos histogramas coloridos por temperatura. Caso contrário, a legenda resolverá para o painel como um todo.

* Usamos `configure_axis(labelAngle=0)` para garantir que nenhum rótulo de eixo seja rotacionado. Isso ajuda a garantir o alinhamento adequado entre os gráficos de dispersão (_scatter plots_) no SPLOM e os histogramas por mês à direita.

_Tente remover ou modificar qualquer um desses ajustes e veja como o layout do painel responde!_

Este painel pode ser reutilizado para mostrar dados de outros locais ou de outros conjuntos de dados. _Atualize o painel para mostrar padrões climáticos para Nova York em vez de Seattle._


## 5.7 Resumo
---


Para obter mais detalhes sobre a composição de múltiplas visualizações, incluindo controle sobre espaçamento de subplotagens e rótulos de cabeçalho, consulte a [documentação do Altair Compound Charts](https://altair-viz.github.io/user_guide/compound_charts.html).

Agora que vimos como compor múltiplas visualizações, estamos prontos para colocá-las em ação. Além de apresentar dados estaticamente, múltiplas visualizações podem permitir exploração multidimensional interativa. Por exemplo, usando seleções vinculadas , podemos destacar pontos em uma visualização para ver valores correspondentes destacados em outras visualizações.

No próximo capítulo, examinaremos como criar seleções interativas para plotagens individuais e composições com múltiplas visualizações.